# Part I. ETL Pipeline for Pre-Processing the Files

#### Import relevant Python packages 

In [21]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [22]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [23]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [25]:
# Import the Cluster class
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # Establish connection and begin executing queries with a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [26]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS semispaqify
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [27]:
try:
    session.set_keyspace('semispaqify')
except Exception as e:
    print(e)

## Queries to ask three questions of the data using Apache Cassandra


### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Explanation: The primary key is created on session_id and item_session to have our partition key on session_id, so that records of the same session are saved in the same partition. With this, our query, with WHERE clause pointing to session_id is optimized for read operation.

In [28]:
# CREATE statement
query = 'CREATE TABLE IF NOT EXISTS song_session_tbl'
query = query + "(session_id int, item_session int, artist_name text, song_title text, song_length float,  \
 PRIMARY KEY (session_id, item_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [29]:
# INSERT statement
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_session_tbl (session_id, item_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        # I did typecasting to match the table model
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))) 

In [30]:
# SELECT statement
query = "SELECT artist_name, song_title, song_length FROM song_session_tbl WHERE session_id = 338 AND item_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Explanation: The primary key is created on both the user_id and session_id, serving as the partition key. Since we are interested in only records that match both conditions as shown in the WHERE clause. The clustering column is item_session since it is the column to be used for sorting.

In [31]:
# CREATE statement
query = 'CREATE TABLE IF NOT EXISTS users_tbl'
query = query + "(user_id int, session_id int, item_session int, artist_name text, song_title text, user_firstname text, user_lastname text, \
PRIMARY KEY ((user_id, session_id), item_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)             

In [32]:
# INSERT statement
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO users_tbl (user_id, session_id, item_session, artist_name, song_title, \
        user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # I did typecasting to match the table model
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 

In [33]:
# SELECT statement
query = "SELECT artist_name, song_title, user_firstname, user_lastname FROM users_tbl \
WHERE user_id = 10 AND session_id = 182 ORDER BY item_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name, row.song_title, row.user_firstname, row.user_lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


#### Explanation: The primary key is created on song_tltle and user_id becausae the WHERE clause in the query points to song_title and user_id would provide uniqueness for partitioning.

In [34]:
# CREATE statement   
query = 'CREATE TABLE IF NOT EXISTS user_song_tbl'
query = query + "(song_title text, user_id int, user_firstname text, user_lastname text, PRIMARY KEY (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)              

In [35]:
# INSERT statement
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO user_song_tbl (song_title, user_id, user_firstname, user_lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        # I did typecasting to match the table model
        session.execute(query, (line[9], int(line[10]), line[1], line[4])) 

In [36]:
# SELECT statement
query = "SELECT user_firstname, user_lastname FROM user_song_tbl WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.user_firstname, row.user_lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [37]:
query = "DROP TABLE song_session_tbl"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query1 = "DROP TABLE users_tbl"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
query2 = "DROP TABLE user_song_tbl"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()